# 3d mitochondria visualizer with synapses
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


# Enter a cellid of interest

In [6]:
# pull all mitos from a cellid of interest
cellid = 648518346349537988

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf

# interesting cellids
# 648518346349530724 microglia with 149 mitos
# 648518346349527319 astrocyte containing the largest mito in the volume and several additional large mitos
# 648518346349537555 apical dendrite likely from a L5 or L6 pyr neuron
# 648518346349537741 pyr neuron with largest mito in the volume for a neuron
# 648518346349508279 oligodendrocyte
# 648518346349539853 one of three neurons in the default view of Neuroglancer
# 648518346349539376 pyr neuron with large apical dendritic arbor
# 648518346349536391 partial pyr neuron with basal dendrite in volume
# 648518346349539934 pyr neuron
# 648518346349525715 astrocyte
# 648518346349516236 pyr spiny dendrite in contact with 648518346349537982
# 648518346349537982 very large spiny dendrite (L5?)
# 648518346349537988 continuation of process as 648518346349537982 but was not continuously segmented

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
1991,1097790,94280,61316,48760,438,61274,48570,425,61356,48992,447,648518346349537988,219511.28,174560.80,17520
4627,974080,1196248,60190,43284,310,59816,42098,278,60824,44608,343,648518346349537988,215480.20,154956.72,12400
12083,978894,295928,60090,44504,310,59806,43926,246,60320,44896,335,648518346349537988,215122.20,159324.32,12400
139497,1092891,63504,61088,46816,373,60880,46700,360,61252,46966,389,648518346349537988,218695.04,167601.28,14920
148610,1107479,2468,61744,51968,541,61724,51934,539,61766,52026,544,648518346349537988,221043.52,186045.44,21640
282265,979279,327576,60362,44586,313,60158,44126,278,60662,45062,333,648518346349537988,216095.96,159617.88,12520
292661,860486,1022976,59274,43340,187,58524,41816,21,59970,44536,311,648518346349537988,212200.92,155157.20,7480
346125,848538,7656,57312,37102,272,57282,37078,270,57344,37126,276,648518346349537988,205176.96,132825.16,10880
392531,972787,28648,58858,42786,120,58814,42638,106,58906,42898,138,648518346349537988,210711.64,153173.88,4800
415739,978900,9452,59852,43662,305,59814,43626,303,59890,43698,309,648518346349537988,214270.16,156309.96,12200


In [7]:
mito_query_mitolist = mito_querydf.mito_id.to_list()
print(f"length: "+str(len(mito_query_mitolist)))
print(f"type: "+str(type(mito_query_mitolist)))
print('')
# print(mito_query_mitolist) # uncomment to print the mito id list

length: 41
type: <class 'list'>



# vtk 3d viewer

In [8]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [9]:
cell_id = cellid

In [10]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [11]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [12]:
len(mito_query_mitolist)

41

In [13]:
mito_query_mitolist[0]

1097790

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [14]:
# download the mito meshes for this cell

for i in range(len(mito_query_mitolist)):
    mito_id = mito_query_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [15]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator = {}
for i in range(len(mito_query_mitolist)):
    mito_seg_id = mito_query_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator)

In [16]:
cell_opac = 0.075
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000194C9B39C30) at 0x00000194CCB2F528>

In [17]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

# Add synapses to visualization
- Download synapse table from https://www.microns-explorer.org/phase1

In [18]:
syn = pd.read_csv('data/221206_pni_synapses_v185.csv')
syn

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,2477272,648518346349538285,648518346349536759,1103,280784,261896,40600,70050,65534,1015,70196,65474,1015,70104,65602,1015
1,4538436,648518346342404863,648518346342414574,62,365832,292808,25800,91472,73196,645,91458,73202,645,91396,73216,645
2,1124039,648518346342921567,648518346345323296,62,229144,155952,48080,57250,39028,1202,57286,38988,1202,57312,38966,1203
3,5245264,648518346342796822,648518346349537255,955,398452,208560,50760,99538,52184,1276,99613,52140,1269,99550,52098,1276
4,6928613,648518346342800412,648518346348190542,718,461868,245212,53560,115462,61350,1335,115467,61303,1339,115468,61296,1335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239270,2167340,648518346348045782,648518346349537840,106,268776,173356,14360,67190,43302,361,67194,43339,359,67216,43364,360
3239271,1944654,648518346347834064,648518346349537840,50,267344,156520,8520,66762,39152,213,66836,39130,213,66874,39134,213
3239272,2749491,648518346347876810,648518346349537840,139,292772,156680,12600,73136,39170,315,73193,39170,315,73218,39160,315
3239273,6924909,648518346348343883,648518346349537840,987,462768,237776,8520,115680,59370,208,115692,59444,213,115628,59430,214


In [19]:
# filter all the pre- and post- synapses for this cell
cell_post_subgraph = syn.query(f'post_root_id=={seg_id}') #this shows all post-syn sites on this seg_id cell
cell_pre_subgraph = syn.query(f'pre_root_id=={seg_id}') #this one shows all pre-syn sites on the axon of seg_id cell

# make an array of xyz synapse positions
postsyn_xyz=cell_post_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
presyn_xyz=cell_pre_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values

In [20]:
len(postsyn_xyz)

159

In [21]:
len(presyn_xyz)

0

In [22]:
# Make a list of the pre_root_ids that form a synapse on this cell
cell_post_subgraph

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
1985529,1353686,648518346344427785,648518346349537988,653,238472,179368,15040,59562,44922,379,59618,44842,376,59656,44914,379
1985538,1521676,648518346344943384,648518346349537988,672,243864,153408,8120,60980,38418,202,60966,38352,203,60906,38364,202
1985540,1384932,648518346349517141,648518346349537988,685,243320,197776,21560,60832,49484,541,60830,49444,539,60836,49390,541
1985542,1526580,648518346345305590,648518346349537988,696,251016,154144,11760,62690,38524,297,62754,38536,294,62768,38592,297
1985543,1384823,648518346342788344,648518346349537988,813,243200,200816,18040,60820,50146,456,60800,50204,451,60848,50228,456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2718374,1559340,648518346341411085,648518346349537988,290,243496,184440,14960,60892,46062,377,60874,46110,374,60810,46086,376
2733343,1132229,648518346344242017,648518346349537988,111,232676,168396,920,58152,42116,22,58169,42099,23,58132,42098,24
2741910,1384585,648518346345646642,648518346349537988,58,242384,198768,19040,60564,49666,476,60596,49692,476,60566,49676,477
2749068,1580308,648518346341401569,648518346349537988,56,247408,199896,19680,61812,49920,492,61852,49974,492,61838,50008,493


In [23]:
# Uncomment if you want to generate a list for copy and pasting
# cell_post_subgraph["pre_root_id"].to_list()

In [24]:
# Make a list of the post_root_ids that receive a synapse from this cell
cell_pre_subgraph

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx


In [25]:
# Uncomment if you want to generate a list for copy and pasting
# cell_pre_subgraph["post_root_id"].to_list()

In [26]:
# settings to visualize pre and post synaptic sites (as dots)
pre_color = (0.2, 0.9, 0.2)
post_color = (0.9, 0.2, 0.2)
pre_opac = 0.25
post_opac = 0.25
pre_size = 250
post_size = 200
presyn_actor = trimesh_vtk.point_cloud_actor(presyn_xyz, size=pre_size, opacity=pre_opac, color=pre_color)  
postsyn_actor = trimesh_vtk.point_cloud_actor(postsyn_xyz, size=post_size, opacity=post_opac, color=post_color)

# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator['presyn_actor'] = presyn_actor
actor_iterator['postsyn_actor'] = postsyn_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)



setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000194C9B33730) at 0x00000194CCB20EE8>

In [27]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

In [28]:
actor_iterator

{'mito_mesh_actor_0': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D26835B0) at 0x00000194CCB2F468>,
 'mito_mesh_actor_1': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D26865D0) at 0x00000194CCB2F408>,
 'mito_mesh_actor_2': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D26837E0) at 0x00000194CCB5BD68>,
 'mito_mesh_actor_3': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D2683C40) at 0x00000194CCB5B108>,
 'mito_mesh_actor_4': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D2684500) at 0x00000194CCB5B0A8>,
 'mito_mesh_actor_5': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D2684960) at 0x00000194CCB5BEE8>,
 'mito_mesh_actor_6': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D26814E0) at 0x00000194CCB5B468>,
 'mito_mesh_actor_7': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D2680360) at 0x00000194CCB5BF48>,
 'mito_mesh_actor_8': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000194D2684DC0) at 0x00